In [ ]:
# Instalación de librerías necesarias
!pip install yfinance wbdata --quiet


In [ ]:
import yfinance as yf
import wbdata
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
# Descargar datos de Yahoo Finance
ticker = 'F'  # Ford como ejemplo
df = yf.download(ticker, start='2015-01-01', end='2025-01-01', interval='1wk')[['Close']].dropna()
df.index = pd.to_datetime(df.index)
df = df.asfreq('W')
df = df.fillna(method='ffill')


In [ ]:
# Definir indicadores del Banco Mundial
indicators = {
    'FP.CPI.TOTL.ZG': 'inflacion',
    'GC.DOD.TOTL.GD.ZS': 'deuda',
    'NE.RSB.GNFS.ZS': 'balanza'
}

# Descargar datos exógenos para EE.UU.
data_date = (datetime.datetime(2015, 1, 1), datetime.datetime(2025, 1, 1))
exogenas = wbdata.get_dataframe(indicators, country='USA', data_date=data_date, convert_date=True)

# Reescalar y preparar
exogenas = exogenas.resample('W').ffill().bfill()


In [ ]:
# Unir series
df_all = df.join(exogenas, how='inner').dropna()
target = df_all['Close']
exog = df_all[['inflacion', 'deuda', 'balanza']]

# Separar entrenamiento y test
train_endog = target[:'2023-12-31']
test_endog = target['2024-01-01':]

train_exog = exog[:'2023-12-31']
test_exog = exog['2024-01-01':]


In [ ]:
# Ajustar modelo ARIMAX
model = sm.tsa.SARIMAX(train_endog, exog=train_exog, order=(1,1,1), seasonal_order=(0,0,0,0))
results = model.fit(disp=False)
print(results.summary())


In [ ]:
# Predicción
forecast = results.forecast(steps=len(test_endog), exog=test_exog)
forecast.index = test_endog.index

# Evaluación
mae = mean_absolute_error(test_endog, forecast)
rmse = np.sqrt(mean_squared_error(test_endog, forecast))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")


In [ ]:
# Visualización
plt.figure(figsize=(12,5))
plt.plot(train_endog.index, train_endog, label='Entrenamiento')
plt.plot(test_endog.index, test_endog, label='Real 2024')
plt.plot(forecast.index, forecast, label='Pronóstico ARIMAX')
plt.legend()
plt.title('Predicción ARIMAX con variables exógenas')
plt.show()
